In [ ]:
# 첫 번째 셀: 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# 두 번째 셀: 데이터 로드 및 전처리
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Data2/ELG_Busan_PoC_per_CA_site_0226_0519.csv')

# 타임스탬프를 datetime으로 변환
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')

# 요일과 시간 정보 추가
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour

# enb_id별 RBused 계산
enb_id_rbused = df.groupby(['enb_id', 'timestamp', 'day_of_week', 'hour', 'Holiday'])['RBused'].sum().reset_index()

# 세 번째 셀: 주간 패턴 계산 (enb_id별)
weekly_pattern = enb_id_rbused.groupby(['enb_id', 'day_of_week', 'hour'])['RBused'].mean().unstack(level=[1, 2])
weekly_pattern.columns = [f'day{day}_hour{hour}' for day, hour in weekly_pattern.columns]

# 네 번째 셀: 새로운 특성 추가 (enb_id별로 계산)
# 2. 시간대별 사용 패턴
peak_hours_usage = enb_id_rbused[enb_id_rbused['hour'].between(9, 18)].groupby('enb_id')['RBused'].mean()
night_hours_usage = enb_id_rbused[enb_id_rbused['hour'].between(0, 5)].groupby('enb_id')['RBused'].mean()
usage_variability = enb_id_rbused.groupby('enb_id')['RBused'].agg(lambda x: x.std() / x.mean())

# 3. 휴일 vs 평일 사용 패턴
holiday_usage = enb_id_rbused[enb_id_rbused['Holiday'] == 1].groupby('enb_id')['RBused'].mean()
workday_usage = enb_id_rbused[enb_id_rbused['Holiday'] == 0].groupby('enb_id')['RBused'].mean()
holiday_workday_ratio = holiday_usage / workday_usage

# 4. 사용자 특성
user_features = df.groupby(['enb_id', 'timestamp']).agg({
    'Usertotal': 'sum',
    'RBused': 'sum'
}).reset_index()

user_features['user_per_RB'] = user_features['Usertotal'] / user_features['RBused']

avg_users = user_features.groupby('enb_id')['Usertotal'].mean()
user_variability = user_features.groupby('enb_id')['Usertotal'].agg(lambda x: x.std() / x.mean())




# 다섯 번째 셀: 특성 집계 및 결합
new_features = pd.DataFrame({
    'peak_hours_usage': peak_hours_usage,
    'night_hours_usage': night_hours_usage,
    'usage_variability': usage_variability,
    'holiday_workday_ratio': holiday_workday_ratio,
    'avg_users': avg_users,
    'user_per_RB': user_features.groupby('enb_id')['user_per_RB'].mean(),
    'user_variability': user_variability,
})

# 기존 weekly_pattern과 새로운 특성들을 결합
combined_features = pd.concat([weekly_pattern, new_features], axis=1)

# 여섯 번째 셀: 데이터 정제 및 전처리
# 무한대 값 처리
combined_features = combined_features.replace([np.inf, -np.inf], np.nan)

# 모든 값이 NaN인 열 제거
combined_features = combined_features.dropna(axis=1, how='all')

# 남은 NaN 값들을 중앙값으로 대체
imputer = SimpleImputer(strategy='median')
combined_features_imputed = pd.DataFrame(imputer.fit_transform(combined_features),
                                         columns=combined_features.columns,
                                         index=combined_features.index)

# 이상치 처리: 99.9 퍼센타일을 초과하는 값들을 99.9 퍼센타일 값으로 대체
for column in combined_features_imputed.columns:
    max_value = combined_features_imputed[column].quantile(0.999)
    combined_features_imputed[column] = combined_features_imputed[column].clip(upper=max_value)

# 일곱 번째 셀: 정규화 및 가중치 적용
# 정규화
scaler = MinMaxScaler()
combined_features_scaled = pd.DataFrame(scaler.fit_transform(combined_features_imputed),
                                        columns=combined_features_imputed.columns,
                                        index=combined_features_imputed.index)

# 가중치 적용
# 전체 가중치의 합이 7:3 비율이 되도록 설정
total_weight = 10
time_features_count = 168  # 7일 * 24시간
other_features_count = len(combined_features_scaled.columns) - time_features_count
time_total_weight = 5
other_total_weight = 5

# 가중치 계산
time_weight = (time_total_weight / time_features_count)
other_weight = (other_total_weight / other_features_count)

# 가중치 적용
weighted_features = combined_features_scaled.copy()
weighted_features.iloc[:, :time_features_count] *= time_weight
weighted_features.iloc[:, time_features_count:] *= other_weight

# 여덟 번째 셀: 클러스터링 수행
# (이전 코드는 그대로 유지)

# 여덟 번째 셀: 클러스터링 수행
# 클러스터링 수행
n_clusters = 6  # 클러스터 수는 필요에 따라 조정 가능
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(weighted_features)

# 결과를 원본 데이터에 추가
combined_features_scaled['cluster'] = cluster_labels
print("클러스터링 완료. 각 클러스터의 크기:")
print(combined_features_scaled['cluster'].value_counts())

# 아홉 번째 셀: 데이터 품질 확인
print("\n데이터 품질 확인:")
print(combined_features_scaled.describe())
print("\n무한대 값 확인:")
print(combined_features_scaled.isin([np.inf, -np.inf]).sum())
print("\nNaN 값 확인:")
print(combined_features_scaled.isna().sum())

# 열 번째 셀: 클러스터링 결과 시각화
# 두 개의 특성 선택 (예: 첫 번째와 두 번째 비시간대 특성)
feature1 = combined_features_scaled.columns[168]
feature2 = combined_features_scaled.columns[169]

plt.figure(figsize=(10, 8))
scatter = plt.scatter(combined_features_scaled[feature1], combined_features_scaled[feature2],
                      c=cluster_labels, cmap='viridis')
plt.colorbar(scatter)
plt.title(f'Clustering Results ({feature1} vs {feature2})')
plt.xlabel(feature1)
plt.ylabel(feature2)
plt.show()

# 열한 번째 셀: 클러스터 중심 시각화
plt.figure(figsize=(15, 10))
for i in range(n_clusters):
    cluster_center = kmeans.cluster_centers_[i][:168] / time_weight  # 가중치 제거
    plt.plot(cluster_center, label=f'Cluster {i}')
plt.title("Cluster Centers: Weekly Patterns")
plt.xlabel("Time (hours)")
plt.ylabel("Normalized Usage")
plt.legend()
plt.show()

# 열두 번째 셀: 클러스터별 주요 특성 분석
cluster_means = combined_features_scaled.groupby('cluster').mean()
for cluster in range(n_clusters):
    print(f"\nCluster {cluster} 주요 특성:")
    print(cluster_means.iloc[cluster, 168:].nlargest(5))  # 상위 5개 특성 출력

# 열세 번째 셀: 클러스터별 시간대 패턴 시각화 (동적으로 조정)
fig, axes = plt.subplots(figsize=(20, 15), nrows=(n_clusters + 1) // 2, ncols=2)
fig.suptitle("Cluster-wise Average Patterns", fontsize=16)

for i in range(n_clusters):
    row = i // 2
    col = i % 2
    ax = axes[row, col] if n_clusters > 1 else axes

    cluster_data = combined_features_scaled[combined_features_scaled['cluster'] == i].iloc[:, :168]
    cluster_mean = cluster_data.mean()
    ax.plot(cluster_mean)
    ax.set_title(f'Cluster {i} Average Pattern')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Normalized Usage')

# 사용하지 않는 서브플롯 제거
if n_clusters % 2 != 0:
    fig.delaxes(axes[-1, -1])

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# 새로운 셀: RBused 히트맵 생성
# 일주일간의 RBused 데이터 준비 (enb_id별 합계 사용)
weekly_rbused = enb_id_rbused.groupby(['day_of_week', 'hour'])['RBused'].mean().unstack()

# 히트맵 그리기
plt.figure(figsize=(12, 8))
sns.heatmap(weekly_rbused, cmap='YlOrRd')
plt.title('Average RBused by Day and Hour (Aggregated by enb_id)')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.show()

In [ ]:
# 전체 데이터의 최소값과 최대값 계산
vmin = 0
vmax = 250

# 새로운 셀: 클러스터별 RBused 히트맵 생성
plt.figure(figsize=(20, 5 * ((n_clusters + 1) // 2)))
for i in range(n_clusters):
    cluster_data = enb_id_rbused[enb_id_rbused['enb_id'].isin(combined_features_scaled[combined_features_scaled['cluster'] == i].index)]
    cluster_weekly_rbused = cluster_data.groupby(['day_of_week', 'hour'])['RBused'].mean().unstack()
    plt.subplot((n_clusters + 1) // 2, 2, i + 1)
    sns.heatmap(cluster_weekly_rbused, cmap='YlOrRd', vmin=vmin, vmax=vmax)
    plt.title(f'Cluster {i}: Average RBused by Day and Hour')
    plt.xlabel('Hour of Day')
    plt.ylabel('Day of Week')
plt.tight_layout()
plt.show()

# 전체 데이터에 대한 히트맵 (기존 코드)
weekly_rbused = enb_id_rbused.groupby(['day_of_week', 'hour'])['RBused'].mean().unstack()
plt.figure(figsize=(12, 8))
sns.heatmap(weekly_rbused, cmap='YlOrRd', vmin=vmin, vmax=vmax)
plt.title('Overall: Average RBused by Day and Hour (Aggregated by enb_id)')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.show()

# 클러스터별 통계 출력
for i in range(n_clusters):
    cluster_data = enb_id_rbused[enb_id_rbused['enb_id'].isin(combined_features_scaled[combined_features_scaled['cluster'] == i].index)]
    print(f"\nCluster {i} Statistics:")
    print(cluster_data['RBused'].describe())
    print(f"Number of enb_ids in Cluster {i}: {cluster_data['enb_id'].nunique()}")

In [ ]:
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm
import branca

# 클러스터링 결과를 enb_id에 매핑
enb_id_cluster = pd.DataFrame({'enb_id': combined_features.index, 'cluster': cluster_labels})

# 원본 데이터에 클러스터 정보 추가
df_with_clusters = df.merge(enb_id_cluster, on='enb_id', how='left')

# 지도 시각화
center_lat = df_with_clusters['ru_svc_lat_val'].mean()
center_lng = df_with_clusters['ru_svc_lng_val'].mean()
m = folium.Map(location=[center_lat, center_lng], zoom_start=12)

colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF',
          '#800000', '#008000', '#000080', '#808000', '#800080', '#008080',
          '#FFA500', '#FFC0CB', '#40E0D0', '#FF69B4', '#7B68EE', '#98FB98', '#DDA0DD']

# enbid_pci별로 그룹화하여 평균 위치와 클러스터 정보 계산
enbid_pci_group = df_with_clusters.groupby('enbid_pci').agg({
    'ru_svc_lat_val': 'mean',
    'ru_svc_lng_val': 'mean',
    'cluster': 'first',  # 각 enbid_pci에 대해 첫 번째 클러스터 값 사용
    'RBused': 'mean'
}).reset_index()

for idx, row in enbid_pci_group.iterrows():
    folium.CircleMarker(
        location=[row['ru_svc_lat_val'], row['ru_svc_lng_val']],
        radius=10,
        popup=f"enbid_pci: {row['enbid_pci']}, Cluster: {row['cluster']}, Avg RBused: {row['RBused']:.2f}",
        color=colors[int(row['cluster']) % len(colors)],
        fill=True,
        fillColor=colors[int(row['cluster']) % len(colors)],
        fillOpacity=0.7,
        stroke=True,
        weight=2
    ).add_to(m)

# 범례 추가
legend_html = '''
     <div style="position: fixed;
     top: 50px; right: 50px; width: 120px; height: 180px;
     border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp; <b>Cluster Legend</b> <br>
     '''

for i in range(n_clusters):
    legend_html += f'<i class="fa fa-circle fa-1x" style="color:{colors[i]}"></i>&nbsp; Cluster {i}<br>'

legend_html += '</div>'

m.get_root().html.add_child(folium.Element(legend_html))

# 지도 저장
m.save("/content/drive/MyDrive/Colab_Notebooks/cluster_map_enbid_pci_with_legend.html")

print("클러스터 지도가 '/content/drive/MyDrive/Colab_Notebooks/cluster_map_enbid_pci_with_legend.html'에 저장되었습니다.")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_enbid_pci_traffic(df, enbid_pci):
    # 해당 enbid_pci의 데이터만 필터링
    enbid_data = df[df['enbid_pci'] == enbid_pci]
    
    if enbid_data.empty:
        print(f"No data found for enbid_pci: {enbid_pci}")
        return
    
    # 일주일 동안의 평균 RBused 계산
    weekly_rbused = enbid_data.groupby(['day_of_week', 'hour'])['RBused'].mean().reset_index()
    
    # day_of_week와 hour를 이용해 연속적인 시간 인덱스 생성
    weekly_rbused['time_index'] = weekly_rbused['day_of_week'] * 24 + weekly_rbused['hour']
    weekly_rbused = weekly_rbused.sort_values('time_index')
    
    # 그래프 생성
    plt.figure(figsize=(20, 8))
    
    plt.plot(weekly_rbused['time_index'], weekly_rbused['RBused'], marker='o', linestyle='-', markersize=4)
    
    plt.title(f'Weekly RBused Traffic for enbid_pci: {enbid_pci}', fontsize=16)
    plt.xlabel('Day and Hour', fontsize=12)
    plt.ylabel('Average RBused', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # X축 레이블 설정
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    xticks = range(0, 168, 24)
    plt.xticks(xticks, days)
    
    # 각 날짜 변경 시점에 수직선 추가
    for day in range(1, 7):
        plt.axvline(x=day*24, color='r', linestyle='--', alpha=0.5)
    
    # Y축 범위 설정 (최소값의 90%부터 최대값의 110%까지)
    y_min = weekly_rbused['RBused'].min() * 0.9
    y_max = weekly_rbused['RBused'].max() * 1.1
    plt.ylim(y_min, y_max)
    
    plt.tight_layout()
    plt.show()
    
    # 기본 통계 출력
    print(f"\nBasic Statistics for enbid_pci {enbid_pci}:")
    print(enbid_data['RBused'].describe())

# 사용 예시:
visualize_enbid_pci_traffic(df, '34804_252')